In [2]:
import * as torch from "../index.ts";

const a = new torch.Tensor([1, 2, 3], [3, 1], true);
a._name = "a_tensor";
const b = new torch.Tensor([4, 5, 6], [3, 1], true);
b._name = "b_tensor";
const c = torch.add(a, b);
c._name = "c_tensor";
console.log("a:", a.toString());
console.log("b:", b.toString());
console.log("c:", c.toString());

a: Tensor(shape=[3,1])
b: Tensor(shape=[3,1])
c: Tensor(shape=[3,1])


In [3]:
const MN = new torch.Tensor(
  [
    2, 3,
    3, 7
  ]
, [2, 2])
MN.toString()

"Tensor(shape=[2,2])"

In [4]:
c.requires_grad

true

In [5]:
const d = torch.mul(a, c);
d._name = "d_tensor";
const e = torch.matmul(d, b.transpose());
e._name = "e_tensor";

"e_tensor"

In [6]:
d.backward(new torch.Tensor([0.001, 0.002, 0.003], [3, 1]));
// e.backward(new torch.Tensor([0.2, 0.8, 0.7], [1, 3]));

In [7]:
function logPrevs(tensor: torch.Tensor, indent: string = "") {
  if (tensor._prev) {
    for (let i = 0; i < tensor._prev.length; i++) {
      const prev = tensor._prev[i];
      console.log(`${indent}Prev Tensor: ${prev.toString()} (Name: ${prev?._name}), Grad: ${prev._grad ? prev._grad.toString() : "None"}`);
      logPrevs(prev, indent + "|  ");
    }
  }
}

function logTree(tensor: torch.Tensor) {
  console.log("Gradient computation graph:");
  console.log(`Tensor: ${tensor.toString()} (Name: ${tensor?._name})`);
  logPrevs(tensor, "|  ");
}

logTree(d);

Gradient computation graph:
Tensor: Tensor(shape=[3,1]) (Name: d_tensor)
|  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: Tensor(shape=[3,1])
|  Prev Tensor: Tensor(shape=[3,1]) (Name: c_tensor), Grad: Tensor(shape=[3,1])
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: Tensor(shape=[3,1])
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: b_tensor), Grad: Tensor(shape=[3,1])


In [8]:
logTree(e);

Gradient computation graph:
Tensor: Tensor(shape=[3,3]) (Name: e_tensor)
|  Prev Tensor: Tensor(shape=[3,1]) (Name: d_tensor), Grad: Tensor(shape=[3,1])
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: Tensor(shape=[3,1])
|  |  Prev Tensor: Tensor(shape=[3,1]) (Name: c_tensor), Grad: Tensor(shape=[3,1])
|  |  |  Prev Tensor: Tensor(shape=[3,1]) (Name: a_tensor), Grad: Tensor(shape=[3,1])
|  |  |  Prev Tensor: Tensor(shape=[3,1]) (Name: b_tensor), Grad: Tensor(shape=[3,1])
|  Prev Tensor: Tensor(shape=[1,3]) (Name: undefined), Grad: None


## Simple Neural Network

In [9]:
let x = new torch.Tensor([7, 8, 9,5],[1,4]);
x._name = "Input (x)"

let W_1 = torch.Tensor.randn([4,8], true);
W_1._name = "W_1"
let B_1 = torch.Tensor.randn([1,8], true);
B_1._name = "W_2"

console.log(x.shape, W_1.shape, B_1.shape)

[ 1, 4 ] [ 4, 8 ] [ 1, 8 ]


In [10]:
let h_1 = torch.add(torch.matmul(x, W_1), B_1);
h_1._name = "h_1"

let W_2 = torch.Tensor.randn([8,2], true);
W_2._name = "W_2"
let B_2 = torch.Tensor.randn([1,2], true);
B_2._name = "B_2"

console.log(h_1.shape, W_2.shape, B_2.shape)

[ 1, 8 ] [ 8, 2 ] [ 1, 2 ]


In [11]:
let h_2 = torch.add(torch.matmul(h_1, W_2), B_2);
h_2._name = "h_2"

"h_2"

In [12]:
h_2

Tensor {
  data: Float32Array(2) [ -10.167489051818848, -73.34496307373047 ],
  shape: [ 1, 2 ],
  requires_grad: true,
  _grad: undefined,
  _prev: [
    Tensor {
      data: Float32Array(2) [ -8.996169090270996, -71.64459991455078 ],
      shape: [ 1, 2 ],
      requires_grad: true,
      _grad: undefined,
      _prev: [
        Tensor {
          data: [Float32Array],
          shape: [Array],
          requires_grad: true,
          _grad: undefined,
          _prev: [Array],
          _name: "h_1",
          _backward: [Function (anonymous)],
          device: "cpu"
        },
        Tensor {
          data: [Float32Array],
          shape: [Array],
          requires_grad: true,
          _grad: undefined,
          _prev: [],
          _name: "W_2",
          _backward: undefined,
          device: "cpu"
        }
      ],
      _name: undefined,
      _backward: [Function (anonymous)],
      device: "cpu"
    },
    Tensor {
      data: Float32Array(2) [ -1.171319603919983, -1

In [13]:
logTree(h_2);

Gradient computation graph:
Tensor: Tensor(shape=[1,2]) (Name: h_2)
|  Prev Tensor: Tensor(shape=[1,2]) (Name: undefined), Grad: None
|  |  Prev Tensor: Tensor(shape=[1,8]) (Name: h_1), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: undefined), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[1,4]) (Name: Input (x)), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[4,8]) (Name: W_1), Grad: None
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: W_2), Grad: None
|  |  Prev Tensor: Tensor(shape=[8,2]) (Name: W_2), Grad: None
|  Prev Tensor: Tensor(shape=[1,2]) (Name: B_2), Grad: None


In [14]:
h_2.backward(new torch.Tensor([0.1, 0.3], [1, 2]));

In [15]:
logTree(h_2);

Gradient computation graph:
Tensor: Tensor(shape=[1,2]) (Name: h_2)
|  Prev Tensor: Tensor(shape=[1,2]) (Name: undefined), Grad: Tensor(shape=[1,2])
|  |  Prev Tensor: Tensor(shape=[1,8]) (Name: h_1), Grad: Tensor(shape=[1,8])
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: undefined), Grad: Tensor(shape=[1,8])
|  |  |  |  Prev Tensor: Tensor(shape=[1,4]) (Name: Input (x)), Grad: None
|  |  |  |  Prev Tensor: Tensor(shape=[4,8]) (Name: W_1), Grad: Tensor(shape=[4,8])
|  |  |  Prev Tensor: Tensor(shape=[1,8]) (Name: W_2), Grad: Tensor(shape=[1,8])
|  |  Prev Tensor: Tensor(shape=[8,2]) (Name: W_2), Grad: Tensor(shape=[8,2])
|  Prev Tensor: Tensor(shape=[1,2]) (Name: B_2), Grad: Tensor(shape=[1,2])


In [16]:
// import * as nn from "https://raw.githubusercontent.com/etornam45/torch-ts/refs/heads/main/dist/bundle.ts"


In [1]:
import { WebGPU } from "../core/backends/webgpu.ts";
import { CPU } from "../core/backends/cpu.ts";
import * as nn from "../index.ts";

let size = 2000;

// Demo: manual WebGPU add (not integrated into ops)
async function demo_webgpu_add() {
  try {
    await WebGPU.init();
  } catch (_e) {
    console.log("WebGPU not available, skipping demo.");
    return;
  }
  const a = nn.Tensor.randn([size, size], false, "webgpu");
  a._name = "A";
  const b = nn.Tensor.randn([size, size], false, "webgpu");
  b._name = "B";
  const out = WebGPU.matmul(WebGPU.add(WebGPU.matmul(WebGPU.matmul(WebGPU.add(a, b), b), a), b), a);
  const data = await WebGPU.readTensorData(out);
  // console.log("WebGPU add result:", Array.from(data));
}

async function demo_cpu_add() {
  const a = nn.Tensor.randn([size, size], false, "cpu");
  a._name = "A"
  const b = nn.Tensor.randn([size, size], false, "cpu");
  b._name = "B"
  const out = CPU.matmul(CPU.add(CPU.matmul(CPU.matmul(CPU.add(a, b), b), a), b), a);
  // console.log("CPU add result:", Array.from(out.data));
}

let st = Date.now();
await demo_cpu_add();
console.log("CPU add time (ms):", Date.now() - st);

st = Date.now();
await demo_webgpu_add();
console.log("WebGPU add time (ms):", Date.now() - st);

CPU add time (ms): 131879
WebGPU add time (ms): 2206
